<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/03_Regresi%C3%B3n_Lineal/EMAE_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
import json

import requests
from requests.auth import HTTPBasicAuth

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/alphacast.json", 'r') as f:
    api_token= json.load(f)

Mounted at /content/drive


In [3]:
# Definir user key para la API
user_key = api_token["key"]

In [40]:
# EMAE
emae = requests.get('https://api.alphacast.io/datasets/5331/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('emae.csv','wb').write(emae.content)
emae = pd.read_csv('emae.csv',delimiter = ',')

In [41]:
emae.set_index('Date', inplace = True)
emae = emae.loc['2016-02-01':]
y=emae['Emae - current_prices_mom']

In [14]:
# IPI
ipi = requests.get('https://api.alphacast.io/datasets/5679/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('ipi.csv','wb').write(ipi.content)
ipi = pd.read_csv('ipi.csv',delimiter = ',')
     

ipi['Date'] = pd.to_datetime(ipi['Date'])
ipi.set_index('Date', inplace = True)
ipi = ipi['IPI - sa_MoM']
ipi.columns = ['IPI - sa_mom']
ipi = ipi.loc['2016-02-01':]

In [15]:
# ISAC
isac = requests.get('https://api.alphacast.io/datasets/5565/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('isac.csv','wb').write(isac.content)
isac = pd.read_csv('isac.csv',delimiter = ',')
     

isac['Date'] = pd.to_datetime(isac['Date'])
isac.set_index('Date', inplace = True)
isac = isac['Nivel general - sa_MoM']
isac.columns = ['Nivel general - sa_MoM']
isac = isac.loc['2016-02-01':]

In [24]:
isac=pd.Series(isac, name='ISAC')
ipi=pd.Series(ipi, name='IPI')

In [25]:
X=pd.concat([ipi,isac],axis=1)

In [47]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X[:-1], y)
Yest = lr.predict(X)

In [52]:
print('DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE')
print()
print('Valor de la pendiente o coeficiente "a":')
print(lr.coef_)
print('Valor de la intersección o coeficiente "b":')
print(lr.intercept_)
print()
print('La ecuación del modelo es igual a:')
print('y = ', lr.coef_, 'X + ', lr.intercept_)
print()
print('Precisión del modelo:')
print(lr.score(X[:-1], y))

DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE

Valor de la pendiente o coeficiente "a":
[-0.08535158  0.18368456]
Valor de la intersección o coeficiente "b":
0.08024655076279444

La ecuación del modelo es igual a:
y =  [-0.08535158  0.18368456] X +  0.08024655076279444

Precisión del modelo:
0.18760966342974839


In [62]:
lr.coef_[0]

-0.08535158499818499

In [64]:
emae_predict=X[-1:]['IPI']*lr.coef_[0]+X[-1:]['ISAC']*lr.coef_[1]+lr.intercept_

In [65]:
emae_predict

Date
2022-10-01   -0.663082
dtype: float64